# Hybrid Model with ResNet50 and VGG16 for Image Classification

In [ ]:

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, models, transforms
from torch.optim.lr_scheduler import StepLR
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc, f1_score

# Check if CUDA (GPU) is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:

# Define the base path
base_path = r"E:\UESTC 2024\Ehtisham Paper\2016 binary\Preprocessed_Data"

# Directories for training and testing data
train_dir = os.path.join(base_path, 'train')
test_dir = os.path.join(base_path, 'test')

# Data augmentation and normalization for training and testing datasets
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(30),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.ToTensor(),
        transforms.RandomErasing(p=0.5),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
}


In [ ]:

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=data_transforms['train'])
test_dataset = datasets.ImageFolder(test_dir, transform=data_transforms['test'])

# Split train data into train and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_data, val_data = random_split(train_dataset, [train_size, val_size])

# Dataloaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=8, pin_memory=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False, num_workers=8, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=8, pin_memory=True)


In [ ]:

# Define Hybrid Model with Attention
class HybridModel(nn.Module):
    def __init__(self):
        super(HybridModel, self).__init__()
        self.resnet50 = models.resnet50(pretrained=True)
        self.vgg16 = models.vgg16(pretrained=True)
        
        self.resnet50 = nn.Sequential(*list(self.resnet50.children())[:-1])
        self.vgg16 = nn.Sequential(*list(self.vgg16.children())[:-1])
        
        self.fc1 = nn.Linear(2048 + 25088, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        resnet_out = self.resnet50(x)
        resnet_out = resnet_out.view(resnet_out.size(0), -1)
        
        vgg_out = self.vgg16(x)
        vgg_out = vgg_out.view(vgg_out.size(0), -1)
        
        combined_features = torch.cat((resnet_out, vgg_out), dim=1)
        
        x = self.fc1(combined_features)
        x = torch.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = torch.relu(x)
        x = self.dropout(x)
        x = self.fc4(x)
        return torch.sigmoid(x)


In [ ]:

# Train and Evaluate the model
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(HybridModel().parameters(), lr=1e-5)
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)

best_model = HybridModel().to(device)

# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=100):
    model.train()
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels.float())
            loss.backward()
            optimizer.step()
        scheduler.step()
    return model

best_model = train_model(best_model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=100)


In [ ]:

# Save the trained model
torch.save(best_model.state_dict(), "model.pth")
print("Model saved as model.pth")


In [ ]:

# Run visualizations
plot_tsne_features(best_model, train_dataset)
plot_confusion_matrix(best_model, test_loader)
plot_roc_curve(best_model, test_loader)
